In [1]:
import polars as pl

import seaborn as sns
import matplotlib.pyplot as plt

sns.set()

In [2]:
df1 = pl.read_csv("ingested.csv")
df2 = pl.read_csv("ingested_2015.csv")
df = df1.vstack(df2)

df_filter = df.filter(
    pl.col("year_of_sale").ge(2015)
)
display(df_filter.sample())
print(list(df_filter.columns))
df_vol =   df_filter.group_by("year_of_sale").agg(pl.col("address").len().alias("count_txns")).sort("year_of_sale")

address,lease_remaining,minimum_floor,maximum_floor,built_year,sqft,sqm,flat_type,road,town,year_of_sale,month_of_sale,months_ago,price,psf,band_name,price_is_geq_one_million
str,i64,i64,i64,i64,i64,i64,str,str,str,i64,i64,i64,i64,f64,str,i64
"""332 SERANGOON AVENUE 3""",61,10,12,1986,904,84,"""4 ROOM""","""SERANGOON AVENUE 3""","""SERANGOON""",2023,1,23,645000,713.49,"""60 - 64""",0


['address', 'lease_remaining', 'minimum_floor', 'maximum_floor', 'built_year', 'sqft', 'sqm', 'flat_type', 'road', 'town', 'year_of_sale', 'month_of_sale', 'months_ago', 'price', 'psf', 'band_name', 'price_is_geq_one_million']


In [3]:
idx = pl.DataFrame([
    {'quarter': '2015', 'index': 135.6},
    {'quarter': '2015', 'index': 135.0},
    {'quarter': '2015', 'index': 134.6},
    {'quarter': '2015', 'index': 134.8},
    {'quarter': '2016', 'index': 134.7},
    {'quarter': '2016', 'index': 134.7},
    {'quarter': '2016', 'index': 134.7},
    {'quarter': '2016', 'index': 134.6},
    {'quarter': '2017', 'index': 133.9},
    {'quarter': '2017', 'index': 133.7},
    {'quarter': '2017', 'index': 132.8},
    {'quarter': '2017', 'index': 132.6},
    {'quarter': '2018', 'index': 131.6},
    {'quarter': '2018', 'index': 131.7},
    {'quarter': '2018', 'index': 131.6},
    {'quarter': '2018', 'index': 131.4},
    {'quarter': '2019', 'index': 131.0},
    {'quarter': '2019', 'index': 130.8},
    {'quarter': '2019', 'index': 130.9},
    {'quarter': '2019', 'index': 131.5},
    {'quarter': '2020', 'index': 131.5},
    {'quarter': '2020', 'index': 131.9},
    {'quarter': '2020', 'index': 133.9},
    {'quarter': '2020', 'index': 138.1},
    {'quarter': '2021', 'index': 142.2},
    {'quarter': '2021', 'index': 146.4},
    {'quarter': '2021', 'index': 150.6},
    {'quarter': '2021', 'index': 155.7},
    {'quarter': '2022', 'index': 159.5},
    {'quarter': '2022', 'index': 163.9},
    {'quarter': '2022', 'index': 168.1},
    {'quarter': '2022', 'index': 171.9},
    {'quarter': '2023', 'index': 173.6},
    {'quarter': '2023', 'index': 176.2},
    {'quarter': '2023', 'index': 178.5},
    {'quarter': '2023', 'index': 180.4},
    {'quarter': '2024', 'index': 183.7},
    {'quarter': '2024', 'index': 187.9},
    {'quarter': '2024', 'index': 192.9}
])
idx = (
    idx
    .group_by(pl.col("quarter"))
    .agg(pl.col("index").mean().alias("price_index"))
    .with_columns(pl.col("quarter").cast(pl.Int64).alias("year_of_sale")
))

In [4]:
df_g = df_vol.join(idx, on="year_of_sale")

In [5]:
df_g

year_of_sale,count_txns,quarter,price_index
i64,u32,str,f64
2017,20509,"""2017""",133.25
2022,26720,"""2022""",165.85
2019,22186,"""2019""",131.05
2020,23333,"""2020""",133.85
2018,21561,"""2018""",131.575
2015,17780,"""2015""",135.0
2024,27426,"""2024""",188.166667
2021,29087,"""2021""",148.725
2023,25755,"""2023""",177.175
